In [2]:
import pickle
import sys
sys.path.append("../")
import time
import pandas as pd
import numpy as np
from Join_scheme.data_prepare import process_stats_data
from Join_scheme.bound import Bound_ensemble

In [3]:
data_path = "/home/ubuntu/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
model_folder = "/home/ubuntu/data_CE/saved_models"
data, null_values, key_attrs, table_buckets, equivalent_keys, schema, bin_size = process_stats_data(data_path,
                                                            model_folder, 200, "sub_optimal", False)

bucketizing equivalent key group: {'postLinks.RelatedPostId', 'posts.Id', 'votes.PostId', 'tags.ExcerptPostId', 'comments.PostId', 'postHistory.PostId', 'postLinks.PostId'}
bucketizing equivalent key group: {'postHistory.UserId', 'votes.UserId', 'badges.UserId', 'users.Id', 'posts.OwnerUserId', 'comments.UserId'}


In [20]:
for key in data:
    columns = data[key].columns
    columns = [col.replace(".", "__") for col in columns]
    data[key].columns = columns

In [53]:
import numpy as np
import copy

from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from Join_scheme.data_prepare import identify_key_values
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform, construct_table_query


class Factor:
    """
    This the class defines a multidimensional conditional probability.
    """
    def __init__(self, variables, pdfs, equivalent_variables=[]):
        self.variables = variables
        self.equivalent_variables = equivalent_variables
        self.pdfs = pdfs
        self.cardinalities = dict()
        for i, var in enumerate(self.variables):
            self.cardinalities[var] = pdfs.shape[i]
            if len(equivalent_variables) != 0:
                self.cardinalities[equivalent_variables[i]] = pdfs.shape[i]


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, bns, table_buckets, schema, data=None):
        self.bns = bns
        self.table_buckets = table_buckets
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)
        self.data = data

    def parse_query_simple(self, query, use_bn=True):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        query = query.replace(" where ", " WHERE ")
        query = query.replace(" from ", " FROM ")
        query = query.replace(" and ", " AND ")
        query = query.split(";")[0]
        query = query.strip()
        tables_all = {}
        join_cond = []
        table_query = {}
        join_keys = {}
        tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
        for table_str in tables_str.split(","):
            table_str = table_str.strip()
            if " as " in table_str:
                tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
            else:
                tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

        # processing conditions
        conditions = query.split(" WHERE ")[-1].split(" AND ")
        for cond in conditions:
            table, cond, join, join_key = process_condition(cond, tables_all)
            if not join:
                attr = cond[0]
                op = cond[1]
                value = cond[2]
                if "Date" in attr:
                    assert "::timestamp" in value
                    value = timestamp_transorform(value.strip().split("::timestamp")[0])
                if use_bn:
                    if table not in table_query:
                        table_query[table] = dict()
                    construct_table_query(self.bns[table], table_query[table], attr, op, value)
                else:
                    if op == "=":
                        op = "=="
                    if table not in table_query:
                        table_query[table] = [(attr.replace(".", "__"), op, value)]
                    else:
                        table_query[table].append((attr.replace(".", "__"), op, value))
            else:
                join_cond.append(cond)
                for tab in join_key:
                    if tab in join_keys:
                        join_keys[tab].add(join_key[tab])
                    else:
                        join_keys[tab] = set([join_key[tab]])

        return tables_all, table_query, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
            else:
                table_query = {}
            id_attrs, probs = self.bns[table].query_id_prob(table_query, key_attrs)
            print(probs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res
    
    def query_id_prob_from_data(self, table, pandas_query, key_attrs):
        data_len = len(self.data[table])
        if pandas_query:
            temp_data = self.data[table].query(pandas_query)
        else:
            temp_data = self.data[table]
        pandas_id = [key.replace(".", "__") for key in key_attrs]
        if len(pandas_id) == 1:
            id_data = temp_data[pandas_id[0]].values
            bin_len = len(bound_ensemble_old.table_buckets[table].oned_bin_modes[key_attrs[0]])
            probs = np.zeros(bin_len)
            for i in range(bin_len):
                probs[i] = np.sum(id_data == i)
            return key_attrs, probs
        else:
            id_data = temp_data[pandas_id].values
            key1 = key_attrs[0]
            key2 = key_attrs[1]
            ind = self.table_buckets[table].id_attributes.index(key1)
            if ind == 0:
                return_key = [key1, key2]
            else:
                return_key = [key2, key1]
                id_data = id_data[:, [1,0]]
            len1, len2 = self.table_buckets[table].twod_bin_modes[key1].shape
            probs = np.zeros((len1, len2))
            for i in range(len1):
                temp_data = id_data[id_data[:, 0] == i, 1]
                for j in range(len2):
                    probs[i, j] = np.sum(temp_data == i)
            
            return return_key, probs
        
    
    def get_all_id_conidtional_distribution_from_data(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
                pandas_query = ""
                for attr, op, val in table_query:
                    pandas_query += f"{attr} {op} {val} and "
                pandas_query = pandas_query[0:-5]
            else:
                pandas_query = None
            id_attrs, probs = self.query_id_prob_from_data(table, pandas_query, key_attrs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res

    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions"""
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))
        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for i in range(rest_group_cardinalty):
            rest_group_probs_eliminated = []
            rest_group_modes_eliminated = []
            for table in rest_group_tables:

                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated + rest_group_probs_eliminated,
                                                        all_modes_eliminated + rest_group_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes):
        all_probs = np.stack(all_probs, axis=0)
        all_modes = np.stack(all_modes, axis=0)
        multiplier = np.prod(all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        min_number = np.amin(all_probs[:, non_zero_idx]/all_modes[:, non_zero_idx], axis=0)
        multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
        return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality_bound(self, query_str, use_bn=True):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str, use_bn)
        #print(table_queries)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        if use_bn:
            conditional_factors = self.get_all_id_conidtional_distribution(table_queries, join_keys, equivalent_group)
        else:
            conditional_factors = self.get_all_id_conidtional_distribution_from_data(table_queries, 
                                                                                     join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res


In [11]:
model_path = "/home/ubuntu/data_CE/CE_scheme_models/model_stats_sub_optimal_200.pkl"
with open(model_path, "rb") as f:
    bound_ensemble_old = pickle.load(f)

query_file = "/home/ubuntu/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()

In [54]:
bound_ensemble = Bound_ensemble(bound_ensemble_old.bns, table_buckets, bound_ensemble_old.schema, data)


In [55]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = bound_ensemble.get_cardinality_bound(query, use_bn=False)
    
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(max(res/true_card, true_card/res))
    print(f"query {i}: pred: {res}, true: {true_card}, q-error: {max(res/true_card, true_card/res)}, latency {time.time() - t}")

qerror = np.asarray(qerror)
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(qerror, i)}")
print(f"average latency per query is {np.mean(latency)}")
print(f"total estimation time is {np.sum(latency)}")

query 0: pred: 79851.0, true: 79851, q-error: 1.0, latency 0.09432840347290039
query 1: pred: 12112592.0, true: 10220614, q-error: 1.1851139275976963, latency 0.12600326538085938
query 2: pred: 3936063.0, true: 1458075, q-error: 2.6994928244431873, latency 0.14103174209594727
query 3: pred: 3998357.0, true: 1709781, q-error: 2.338519962498121, latency 0.05901765823364258
query 4: pred: 10553388.0, true: 7491903, q-error: 1.4086391668445253, latency 0.12517166137695312
query 5: pred: 961566.0, true: 428612, q-error: 2.2434416208598917, latency 0.055846214294433594
query 6: pred: 59738071.0, true: 55900138, q-error: 1.0686569503638792, latency 0.06830620765686035
query 7: pred: 11099.0, true: 10972, q-error: 1.0115749179730222, latency 0.016680479049682617
query 8: pred: 60665309.0, true: 699302, q-error: 86.75123051271125, latency 0.12284064292907715
query 9: pred: 39738092.0, true: 36337035, q-error: 1.0935975376086684, latency 0.05954313278198242
query 10: pred: 9480.0, true: 9405, q-

query 89: pred: 192550.0, true: 8528, q-error: 22.578564727954973, latency 0.10141515731811523
query 90: pred: 245790.0, true: 25033, q-error: 9.818639395997284, latency 0.13065814971923828
query 91: pred: 40267513.0, true: 36980622, q-error: 1.0888814417453552, latency 0.06609272956848145
query 92: pred: 10430200.0, true: 242937, q-error: 42.933764720894715, latency 0.1471705436706543
query 93: pred: 62528.0, true: 42543, q-error: 1.4697600075218016, latency 0.08132600784301758
query 94: pred: 3648323.0, true: 84481, q-error: 43.18513038434678, latency 0.17125940322875977
query 95: pred: 51340.0, true: 40496, q-error: 1.2677795337811142, latency 0.06202411651611328
query 96: pred: 24185.0, true: 244, q-error: 99.1188524590164, latency 0.11072182655334473
query 97: pred: 8103201468.0, true: 272870860, q-error: 29.696104113132492, latency 0.19098353385925293
query 98: pred: 1537838.0, true: 256660, q-error: 5.991732252785787, latency 0.10208773612976074
query 99: pred: 1478063.0, true: 

query 178: pred: 76724.0, true: 68895, q-error: 1.1136366935191233, latency 0.04940295219421387
query 179: pred: 101471385.0, true: 91539518, q-error: 1.108498135198833, latency 0.08028173446655273
query 180: pred: 215859.0, true: 70700, q-error: 3.053168316831683, latency 0.051894426345825195
query 181: pred: 4749229.0, true: 1984289, q-error: 2.3934159792248004, latency 0.06075739860534668
query 182: pred: 40780.0, true: 18346, q-error: 2.222827864384607, latency 0.03639936447143555
query 183: pred: 13893480.0, true: 3169724, q-error: 4.383182889109588, latency 0.06696486473083496
query 184: pred: 476636.0, true: 345663, q-error: 1.37890372993349, latency 0.04900360107421875
query 185: pred: 12128102.0, true: 10238947, q-error: 1.1845067661742952, latency 0.05693340301513672
query 186: pred: 67481.0, true: 46989, q-error: 1.436102066441082, latency 0.05930638313293457
query 187: pred: 59647.0, true: 41442, q-error: 1.4392886443704453, latency 0.05476975440979004
query 188: pred: 784.

query 263: pred: 2849624.0, true: 1236745, q-error: 2.304132218040097, latency 0.07908296585083008
query 264: pred: 133808.0, true: 130349, q-error: 1.026536452140024, latency 0.06660866737365723
query 265: pred: 16201350.0, true: 15878293, q-error: 1.0203458268467525, latency 0.0813746452331543
query 266: pred: 37268.0, true: 8538, q-error: 4.364956664324198, latency 0.030053138732910156
query 267: pred: 944587.0, true: 190399, q-error: 4.961092232627272, latency 0.038640499114990234
query 268: pred: 68400.0, true: 58524, q-error: 1.168751281525528, latency 0.025483131408691406
query 269: pred: 1096997.0, true: 181951, q-error: 6.029079257602321, latency 0.08357930183410645
query 270: pred: 245952129.0, true: 120985656, q-error: 2.032903214576115, latency 0.09094929695129395
query 271: pred: 1183608.0, true: 659318, q-error: 1.7952004950570135, latency 0.07829761505126953
query 272: pred: 686376.0, true: 51156, q-error: 13.417311752287121, latency 0.040060997009277344
query 273: pred:

query 349: pred: 10897.0, true: 222, q-error: 49.085585585585584, latency 0.04027581214904785
query 350: pred: 3058698133.0, true: 2809685334, q-error: 1.0886265789220937, latency 0.0885918140411377
query 351: pred: 860451.0, true: 52366, q-error: 16.431482259481342, latency 0.0641927719116211
query 352: pred: 813611.0, true: 3672, q-error: 221.57162309368192, latency 0.09338784217834473
query 353: pred: 5573318.0, true: 3829734, q-error: 1.4552754838847815, latency 0.07407498359680176
query 354: pred: 4205.0, true: 120, q-error: 35.041666666666664, latency 0.02510380744934082
query 355: pred: 16020.0, true: 16, q-error: 1001.25, latency 0.05966353416442871
query 356: pred: 381583.0, true: 4930, q-error: 77.40020283975659, latency 0.10366296768188477
query 357: pred: 22703466897.0, true: 21963242042, q-error: 1.0337028956646965, latency 0.13930654525756836
query 358: pred: 321644.0, true: 813, q-error: 395.6260762607626, latency 0.07027697563171387
query 359: pred: 1097727.0, true: 58,

...

query 2217: pred: 19752701.0, true: 906966, q-error: 21.77887704721015, latency 0.18430495262145996
query 2218: pred: 194945.0, true: 9921, q-error: 19.649732889829654, latency 0.12467670440673828
query 2219: pred: 13630151.0, true: 967844, q-error: 14.083004079169784, latency 0.09315371513366699
query 2220: pred: 19752701.0, true: 10107224, q-error: 1.9543151512225316, latency 0.20795106887817383
query 2221: pred: 3844070.0, true: 122537, q-error: 31.370688037082676, latency 0.17694377899169922
query 2222: pred: 2180918.0, true: 11678813, q-error: 5.3549986748699405, latency 0.1976630687713623
query 2223: pred: 718636.0, true: 139845, q-error: 5.138803675497873, latency 0.2176806926727295
query 2224: pred: 12358737.0, true: 1613494, q-error: 7.659611377544633, latency 0.1399548053741455
query 2225: pred: 87163626.0, true: 3618173, q-error: 24.090508110032328, latency 0.09590315818786621
query 2226: pred: 4269730.0, true: 11014660, q-error: 2.5797087872066853, latency 0.167794227600097

query 2300: pred: 107366727.0, true: 9265894, q-error: 11.587303610423344, latency 0.10460329055786133
query 2301: pred: 4725753.0, true: 23808461, q-error: 5.038024839639313, latency 0.18866729736328125
query 2302: pred: 4567783.0, true: 205841, q-error: 22.190831758493207, latency 0.19036054611206055
query 2303: pred: 2830617.0, true: 127416476, q-error: 45.01367581696853, latency 0.17639970779418945
query 2304: pred: 2393670.0, true: 1079491, q-error: 2.2174061664247318, latency 0.19637107849121094
query 2305: pred: 19923359.0, true: 3452526, q-error: 5.770661538826935, latency 0.18442249298095703
query 2306: pred: 36321542.0, true: 413991602, q-error: 11.397963280303463, latency 0.23535442352294922
query 2307: pred: 19578547.0, true: 4154456, q-error: 4.7126620188058315, latency 0.17836356163024902
query 2308: pred: 19923359.0, true: 70129559, q-error: 3.519966638155745, latency 0.17085886001586914
query 2309: pred: 18750437.0, true: 81958485, q-error: 4.371017326156185, latency 0.

query 2382: pred: 17798491.0, true: 2704307, q-error: 6.581534936676938, latency 0.2261340618133545
query 2383: pred: 19923359.0, true: 43970699, q-error: 2.2069922546695064, latency 0.21524357795715332
query 2384: pred: 18229285.0, true: 46537849, q-error: 2.5529168587797053, latency 0.21032261848449707
query 2385: pred: 5632585.0, true: 2342248, q-error: 2.40477737626417, latency 0.10711431503295898
query 2386: pred: 91630950.0, true: 3701478, q-error: 24.755232909664734, latency 0.17185068130493164
query 2387: pred: 4708473.0, true: 11214638, q-error: 2.3817993646772533, latency 0.23294281959533691
query 2388: pred: 4523630.0, true: 118785, q-error: 38.0825019994107, latency 0.17676329612731934
query 2389: pred: 2538396.0, true: 42793812, q-error: 16.85860362212988, latency 0.2133166790008545
query 2390: pred: 2105985.0, true: 473164, q-error: 4.450856362698769, latency 0.16808795928955078
query 2391: pred: 19923359.0, true: 1706513, q-error: 11.674894360605515, latency 0.1839811801

query 2465: pred: 1856453.0, true: 182241, q-error: 10.186802091735668, latency 0.2080371379852295
query 2466: pred: 18877717.0, true: 1785176, q-error: 10.57470916032929, latency 0.15627336502075195
query 2467: pred: 31210089.0, true: 96498890, q-error: 3.09191332328466, latency 0.2477409839630127
query 2468: pred: 16955345.0, true: 975631, q-error: 17.378850200536885, latency 0.23102307319641113
query 2469: pred: 18877717.0, true: 17589523, q-error: 1.073236437395147, latency 0.17804861068725586
query 2470: pred: 17516062.0, true: 14185466, q-error: 1.2347893259199239, latency 0.1845090389251709
query 2471: pred: 48422363.0, true: 524182774, q-error: 10.825220859213335, latency 0.2761962413787842
query 2472: pred: 44426492.0, true: 4681602, q-error: 9.48959181066652, latency 0.20226097106933594
query 2473: pred: 4686423.0, true: 22385875, q-error: 4.776750839606241, latency 0.2300574779510498
query 2474: pred: 2559227.0, true: 67369732, q-error: 26.324250252126912, latency 0.22236251

query 2549: pred: 727229.0, true: 405572, q-error: 1.793094690954997, latency 0.10054993629455566
query 2550: pred: 33593471.0, true: 458451, q-error: 73.2760338618522, latency 0.08674979209899902
query 2551: pred: 1224804.0, true: 87842, q-error: 13.943261765442498, latency 0.08498811721801758
query 2552: pred: 168567.0, true: 12124, q-error: 13.903579676674365, latency 0.11563944816589355
query 2553: pred: 45084.0, true: 60963, q-error: 1.3522092094756455, latency 0.09429621696472168
query 2554: pred: 300524.0, true: 172157, q-error: 1.7456391549573935, latency 0.13162827491760254
query 2555: pred: 33593471.0, true: 438690, q-error: 76.57678770886046, latency 0.09305787086486816
query 2556: pred: 5179277.0, true: 50824, q-error: 101.90612702660161, latency 0.11255645751953125
query 2557: pred: 1299103.0, true: 249157, q-error: 5.213993586373251, latency 0.13487601280212402
query 2558: pred: 8897206.0, true: 659522, q-error: 13.49038546098538, latency 0.14383578300476074
query 2559: p

In [23]:
len(data['users'].query("users__UpVotes >= 0 and users__CreationDate <= 67121"))

3

In [37]:
a = data['users'][["users__Id", "users__UpVotes"]].values
a

array([[  -1, 5007],
       [   2,    3],
       [   0,   19],
       ...,
       [  -1,    0],
       [   1,    0],
       [  -1,    0]])

In [38]:
a[:, [1,0]]

array([[5007,   -1],
       [   3,    2],
       [  19,    0],
       ...,
       [   0,   -1],
       [   0,    1],
       [   0,   -1]])